# k-Nearest Neighbors (Bill Lee)

# Support Vector Machine (Lester Fan)

# k-Means Clustering (Anton Maliev)

### Preprocessing

In [9]:
import numpy as np
from matplotlib import pyplot as plt 
from PIL import Image
import glob
from skimage.io import imread

daisy,dandelion,rose,sunflower,tulip = [],[],[],[],[]

for filename in glob.glob('flowerscomp/daisy/*.jpg'): # for daisies
    im = imread(filename,True).flatten()
    daisy.append(im)    

for filename in glob.glob('flowerscomp/dandelion/*.jpg'): # for dandelion
    im = imread(filename,True).flatten()
    dandelion.append(im)
    
for filename in glob.glob('flowerscomp/rose/*.jpg'): # for roses
    im = imread(filename,True).flatten()
    rose.append(im)
    
for filename in glob.glob('flowerscomp/sunflower/*.jpg'): # for sunflowers
    im = imread(filename,True).flatten()
    sunflower.append(im)
    
for filename in glob.glob('flowerscomp/tulip/*.jpg'): # for tulip
    im = imread(filename,True).flatten()
    tulip.append(im)

In [15]:
daisy_arr=np.asarray(daisy)
dandelion_arr=np.asarray(dandelion)
rose_arr=np.asarray(rose)
sunflower_arr=np.asarray(sunflower)
tulip_arr=np.asarray(tulip)

# daisy = 0, dandelion = 1, rose = 2, sunflower = 3, tulip = 4
